# Supermarket

## Problem Statement 

Given is a supermarket comprising of on-site energy conversion blocks and energy demands. The expected operating horizon of the supermarket is 20 years. The energy demands are listed in Table 1.


|               | Lighting | Refrigeration | Space Heating | 
|---------------|----------|---------------|---------------|
| Demand [kW]   | 200      | 1000          |100            |

Primary energy resources include natural gas and biomass. Prices of the primary energy resources and grid electricity as well as their associated CO2 production is given in Table 2 – it should be noted that 1 [GJ] is approximately equal to 277.78 [KWh].

|                    | Natural Gas | Biomass | Grid Electricity | 
|--------------------|-------------|---------|------------------|
| Price [\$/GJ]      | 8.89        | 9.72    |36.11             |
| CO$_2$ gen. [kg/PJ]| 56          | 100     | 90               |


The on-site energy generation section involves two electricity generation technologies, one heat generation technology, and one co-production technology. The electrical efficiency (ηE), heating efficiency (ηH), minimum capacity if constructed (LB), maximum capacity if constructed (UB), capital cost (CAPEX), and operational cost (OPEX) are give in Table 3. Table 3: Technical and economic parameters of on-site energy generation technologies

| Process            | ηE [%] | ηH [%] | LB [kW] | UB [kW] | Capex [$/kW] | Opex [$/ (kW·year)] |
|--------------------|--------|--------|---------|---------|--------------|---------------------|
| Biomass ST         |   68   |   0    |   100   | 10⁶     |    250       |        15           |
| Natural Gas CHP    |   44   |  28    |   800   | 10⁶     |    500       |        15           |
| Solar PV           |    9   |   0    |    10   |  300    |   2000       |       500           |
| Wind Farm          |   22   |   0    |    10   |  500    |   2000       |      1200           |


Two electricity driven energy conversion technologies and one heat driven energy conversion technologies is given in order to meet the demands of lighting, refrigeration, and space heating. The efficiency (or coefficient of performance, COP), types of energy input, types of utility output, capital cost and operational cost of these technologies are summarized in Table 4.

|                      | Refrigeration      | LED         | Heating      |
|----------------------|--------------------|-------------|--------------|
| **Input [-]**        | Electricity        | Electricity | Heating      |
| **Output [-]**       | Refrigeration      | Lighting    | Space Heating|
| **Efficiency [%]**   | 300 (COP)          | 80          | 85           |
| **Capex [$/kW]**     | 70                 | 10          | 30           |
| **Opex [$/kW·year]** | 4                  | 1           | 3            |


Determine system configurations for optimal cost, energy efficiency, and emission

## Import modules

In [53]:
from energia import *

## Parameters

In [54]:
resource_demand_dict = {
    'Lighting': [100, 200],
    'Refrigeration': [550, 1000],
    'Space Heating': [100, 200],
}

resource_dict = {
    'Price': {'Natural Gas': 8.89, 'Biomass': 9.72, 'Grid Electricity': 36.11},
    'CO2 Generation': {'Natural Gas': 56, 'Biomass': 100, 'Grid Electricity': 90},
}

generation_process_dict = {
    'Biomass ST': {'nE': 68, 'nH': 0, 'LB': 100, 'UB': 10**6, 'Capex': 250, 'Opex': 15},
    'Natural Gas CHP': {
        'nE': 44,
        'nH': 28,
        'LB': 800,
        'UB': 10**6,
        'Capex': 500,
        'Opex': 15,
    },
    'Solar PV': {'nE': 9, 'nH': 0, 'LB': 10, 'UB': 300, 'Capex': 2000, 'Opex': 500},
    'Wind Farm': {'nE': 22, 'nH': 0, 'LB': 10, 'UB': 500, 'Capex': 2000, 'Opex': 1200},
}

consumption_process_dict = {
    'Refrigeration': {'Efficiency': 300, 'Capex': 80, 'Opex': 85},
    'LED': {'Efficiency': 70, 'Capex': 10, 'Opex': 30},
    'Heating': {'Efficiency': 4, 'Capex': 1, 'Opex': 3},
}

## Model

Declare the model

In [55]:
m = Model('supermarket')

m.y10 = Periods(label='10 years')
m.y20 = 2 * m.y10

## Spatiotemporal Disposition

We will conside a single temporal scale representing the 20 year horizon with no discretization. The only location being considered is the Supermarket
It is <u>not</u> required. However, declaring one allows us to label the location.

In [56]:
m.supermarket = Location()

## Units

In [57]:
m.GJ = Unit(label='Giga Joules')
m.PJ = 10**6 * m.GJ
m.kW = Unit(label='kilo Watts')
m.tons = Unit(label='US Tons')

## Resources

Resources can be declared along with thier attributes such as maximum allowed consumption, dischargeablity, base price, etc.

In [58]:
m.usd = Currency(label='$')

In [59]:
m.biomass = Resource(basis=m.GJ, label='Biomass')
m.biomass.consume == True
m.biomass.consume[m.usd.spend] == resource_dict['Price']['Biomass']

--- General Resource Balance for biomass in (supermarket, y20): initializing constraint, adding consume(biomass, supermarket, y20)
    Completed in 0.00010418891906738281 seconds


### Display the program

show(True) will print constraints at every index

In [60]:
m.show()

# Mathematical Program for Program(supermarket)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

## s.t.

### Calculation Constraint Sets

<IPython.core.display.Math object>

### General Resource Balance Constraint Sets

<IPython.core.display.Math object>

In [61]:
m.gridpower = Resource(basis=m.GJ, label='Grid Electricity')
m.gridpower.consume == True
m.gridpower.consume[m.usd.spend] == resource_dict['Price']['Grid Electricity']


m.ng = Resource(basis=m.GJ, label='Natural gas')
m.ng.consume == True
m.ng.consume[m.usd.spend] == resource_dict['Price']['Natural Gas']


m.power = Resource(basis=m.kW, label='Electrical Power')
m.heat = Resource(basis=m.kW, label='Heat Power')

m.co2_vent = Resource(basis=m.tons, label='Carbon dioxide')
m.co2_vent.release == True


m.solar = Resource(basis=m.GJ, label='Solar energy')
m.solar.consume == True
m.wind = Resource(basis=m.GJ, label='Wind energy')
m.wind.consume == True

m.lighting = Resource(basis=m.kW, label='Lighting')
m.lighting.release >= resource_demand_dict['Lighting']

m.refrigeration = Resource(basis=m.kW, label='Refrigeration')
m.refrigeration.release >= resource_demand_dict['Refrigeration']

m.heating = Resource(basis=m.kW, label='Heating')
m.heating.release >= resource_demand_dict['Space Heating']

--- General Resource Balance for gridpower in (supermarket, y20): initializing constraint, adding consume(gridpower, supermarket, y20)
    Completed in 0.00011348724365234375 seconds
--- General Resource Balance for ng in (supermarket, y20): initializing constraint, adding consume(ng, supermarket, y20)
    Completed in 6.246566772460938e-05 seconds
--- General Resource Balance for co2_vent in (supermarket, y20): initializing constraint, adding release(co2_vent, supermarket, y20)
    Completed in 6.318092346191406e-05 seconds
--- General Resource Balance for solar in (supermarket, y20): initializing constraint, adding consume(solar, supermarket, y20)
    Completed in 6.341934204101562e-05 seconds
--- General Resource Balance for wind in (supermarket, y20): initializing constraint, adding consume(wind, supermarket, y20)
    Completed in 6.0558319091796875e-05 seconds
--- General Resource Balance for lighting in (supermarket, y10): initializing constraint, adding release(lighting, superma

In [62]:
m.show()

# Mathematical Program for Program(supermarket)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Processes

Processes convert one resource to another with a particular conversion efficiency while incurring a cost for setup (capital expenditure) and operation (operational expenditure)

In [63]:
m.st = Process(basis=m.PJ, label='Biomass ST')
m.st(-m.biomass) == (
    277.78 * generation_process_dict['Biomass ST']['nE'] / 100
) * m.power + m.co2_vent * resource_dict['CO2 Generation']['Biomass']
# using x sets a binary, making the setting up of the process optional
m.st.capacity.x >= generation_process_dict['Biomass ST']['LB']
m.st.capacity.x <= generation_process_dict['Biomass ST']['UB']
m.st.capacity[m.usd.spend] == generation_process_dict['Biomass ST']['Capex'] * 0.05
m.st.operate[m.usd.spend] == generation_process_dict['Biomass ST']['Opex']


m.chp = Process(basis=m.PJ, label='Biomass ST')
m.chp(-m.ng) == (
    277.78 * generation_process_dict['Natural Gas CHP']['nE'] / 100
) * m.power + (
    277.78 * generation_process_dict['Natural Gas CHP']['nH'] / 100
) * m.heat + m.co2_vent * resource_dict[
    'CO2 Generation'
][
    'Natural Gas'
]
# using x sets a binary, making the setting up of the process optional
m.chp.capacity.x >= generation_process_dict['Natural Gas CHP']['LB']
m.chp.capacity.x <= generation_process_dict['Natural Gas CHP']['UB']
m.chp.capacity[m.usd.spend] == generation_process_dict['Natural Gas CHP'][
    'Capex'
] * 0.05
m.chp.operate[m.usd.spend] == generation_process_dict['Natural Gas CHP']['Opex']


m.pv = Process(basis=m.kW, label='Solar PV')
m.pv(-m.solar) == (277.78 * generation_process_dict['Solar PV']['nE'] / 100) * m.power
m.pv.capacity.x <= generation_process_dict['Solar PV']['UB']
m.pv.capacity.x >= generation_process_dict['Solar PV']['LB']
m.pv.capacity[m.usd.spend] == generation_process_dict['Solar PV']['Capex'] * 0.05
m.pv.operate[m.usd.spend] == generation_process_dict['Solar PV']['Opex']


m.wf = Process(basis=m.kW, label='Wind Farm')
m.wf(-m.wind) == (277.78 * generation_process_dict['Wind Farm']['nE'] / 100) * m.power
m.wf.capacity.x <= generation_process_dict['Wind Farm']['UB']
m.wf.capacity.x >= generation_process_dict['Wind Farm']['LB']
m.wf.capacity[m.usd.spend] == generation_process_dict['Wind Farm']['Capex'] * 0.05
m.wf.operate[m.usd.spend] == generation_process_dict['Wind Farm']['Opex']

m.grid = Process(basis=m.PJ, label='Grid Electricity')
m.grid(-m.gridpower) == 277.78 * m.power + m.co2_vent * resource_dict['CO2 Generation'][
    'Grid Electricity'
]
m.grid.capacity <= 10**5  # no binary needed because no upper bound

--- Binding capacity in domain (st, supermarket, y20)
    Completed in 0.00015687942504882812 seconds
--- Binding capacity in domain (st, supermarket, y20)
    Completed in 9.632110595703125e-05 seconds
--- Binding capacity in domain (chp, supermarket, y20)
    Completed in 0.00010085105895996094 seconds
--- Binding capacity in domain (chp, supermarket, y20)
    Completed in 9.202957153320312e-05 seconds
--- Binding capacity in domain (pv, supermarket, y20)
    Completed in 0.00010037422180175781 seconds
--- Binding capacity in domain (pv, supermarket, y20)
    Completed in 8.749961853027344e-05 seconds
--- Binding capacity in domain (wf, supermarket, y20)
    Completed in 8.845329284667969e-05 seconds
--- Binding capacity in domain (wf, supermarket, y20)
    Completed in 8.153915405273438e-05 seconds
--- Binding capacity in domain (grid, supermarket, y20)
    Completed in 4.363059997558594e-05 seconds


In [64]:
m.show(True)

# Mathematical Program for Program(supermarket)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [74]:
m.program.cons_int_cut = m.grid.capacity.X() + m.wf.capacity.X() == 1

In [76]:
m.show(True)

# Mathematical Program for Program(supermarket)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

###  Functions

<IPython.core.display.Math object>

In [66]:
m.show()

# Mathematical Program for Program(supermarket)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [67]:
m.refrigerator = Process(basis=m.kW, label='Refrigerator')
m.refrigerator.capacity <= 10**5
m.refrigerator(-m.power) == (
    consumption_process_dict['Refrigeration']['Efficiency'] / 100
) * m.refrigeration
m.refrigerator.capacity[m.usd.spend] == consumption_process_dict['Refrigeration'][
    'Capex'
] * 0.05
m.refrigerator.operate[m.usd.spend] == consumption_process_dict['Refrigeration']['Opex']

m.led = Process(basis=m.kW, label='LED')
m.led(-m.power) == (consumption_process_dict['LED']['Efficiency'] / 100) * m.lighting
m.led.capacity[m.usd.spend] == consumption_process_dict['LED']['Capex'] * 0.05
m.led.operate[m.usd.spend] == consumption_process_dict['LED']['Opex']

m.heater = Process(basis=m.kW, label='Heater')
m.heater(-m.heat) == (
    consumption_process_dict['Heating']['Efficiency'] / 100
) * m.heating
m.heater.capacity[m.usd.spend] == consumption_process_dict['Heating']['Capex'] * 0.05
m.heater.operate[m.usd.spend] == consumption_process_dict['Heating']['Opex']

--- Binding capacity in domain (refrigerator, supermarket, y20)
    Completed in 9.655952453613281e-05 seconds


## Locate operations

Locations serve as aggregations of scenarios

In [68]:
m.supermarket.locate(m.st, m.chp, m.pv, m.wf, m.grid, m.refrigerator, m.led, m.heater)

--- Assuming operation of st is bound by capacity in (supermarket, y20)
--- Binding operate in domain (st, supermarket, y20)
    Completed in 0.00012421607971191406 seconds
--- General Resource Balance for biomass in (supermarket, y20): adding expend(biomass, supermarket, y20, operate, st)
    Completed in 6.937980651855469e-05 seconds
--- General Resource Balance for power in (supermarket, y20): initializing constraint, adding produce(power, supermarket, y20, operate, st)
    Completed in 6.175041198730469e-05 seconds
--- General Resource Balance for co2_vent in (supermarket, y20): adding produce(co2_vent, supermarket, y20, operate, st)
    Completed in 6.747245788574219e-05 seconds
--- Assuming operation of chp is bound by capacity in (supermarket, y20)
--- Binding operate in domain (chp, supermarket, y20)
    Completed in 7.009506225585938e-05 seconds
--- General Resource Balance for ng in (supermarket, y20): adding expend(ng, supermarket, y20, operate, chp)
    Completed in 6.79492

## Formulation

Various constraints are need for the formulating the mathematical programming model. Here we include the cost, production, resource balance, demand, inventory, and network constraints.

In [69]:
m.show(category='General Resource Balance')

# Mathematical Program for Program(supermarket)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Optimal Cost 

In [70]:
m.usd.spend.opt()

--- Mapping spend: (usd, supermarket, y20, consume, biomass) → (usd, supermarket, y20)
    Completed in 0.000s
--- Mapping spend: (usd, supermarket, y20, consume, gridpower) → (usd, supermarket, y20)
    Completed in 0.000s
--- Mapping spend: (usd, supermarket, y20, consume, ng) → (usd, supermarket, y20)
    Completed in 0.000s
--- Mapping spend: (usd, supermarket, y20, capacity, st) → (usd, supermarket, y20)
    Completed in 0.000s
--- Mapping spend: (usd, supermarket, y20, operate, st) → (usd, supermarket, y20)
    Completed in 0.000s
--- Mapping spend: (usd, supermarket, y20, capacity, chp) → (usd, supermarket, y20)
    Completed in 0.000s
--- Mapping spend: (usd, supermarket, y20, operate, chp) → (usd, supermarket, y20)
    Completed in 0.000s
--- Mapping spend: (usd, supermarket, y20, capacity, pv) → (usd, supermarket, y20)
    Completed in 0.000s
--- Mapping spend: (usd, supermarket, y20, operate, pv) → (usd, supermarket, y20)
    Completed in 0.000s
--- Mapping spend: (usd, supe

In [71]:
m.show(True)

# Mathematical Program for Program(supermarket)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

###  Functions

<IPython.core.display.Math object>

In [45]:
m.usd.spend.sol()

In [46]:
m.show(category='Flow')

# Mathematical Program for Program(supermarket)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

###  Function Sets

<IPython.core.display.Math object>

In [47]:
m.capacity.sol()

In [48]:
m.release.sol()

In [49]:
m.operate.sol()

### Optimal Emissions

In [50]:
m.co2_vent.release.opt()

--- Generating Program(supermarket).mps
--- Creating gurobi model for Program(supermarket)
Read MPS format model from file Program(supermarket).mps
Reading time = 0.00 seconds
PROGRAM(SUPERMARKET): 88 rows, 82 columns, 188 nonzeros
--- Optimizing Program(supermarket) using gurobi
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 88 rows, 82 columns and 188 nonzeros
Model fingerprint: 0xfa29359b
Variable types: 78 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+05]
Presolve removed 9 rows and 9 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 24 available proce

### Optimal Energy Efficiency 

In the formulation, the objective can be skipped and a bespoke objective can be introduced as shown below

In [51]:
m._.lb(sum(m._.consume))

--- Generating Program(supermarket).mps
--- Creating gurobi model for Program(supermarket)
Read MPS format model from file Program(supermarket).mps
Reading time = 0.00 seconds
PROGRAM(SUPERMARKET): 88 rows, 82 columns, 188 nonzeros
--- Optimizing Program(supermarket) using gurobi
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 88 rows, 82 columns and 188 nonzeros
Model fingerprint: 0x52560f5d
Variable types: 78 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+05]
Presolve removed 9 rows and 9 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 24 available proce

In [52]:
m.solution

{}